In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import nibabel as nib
from scipy import ndimage
from tqdm import tqdm
import os
import random
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, roc_auc_score

# **Preprocessing for clinical variables**

## Specify the paths to csv files for clinical variables

The csv files should be created separately according to the following classification method.
*   training and validation, expansion
*   training and validation, no expansion
*   test, expansion
*   test, no expansion

Only the following variables should be included in the csv files.
*   Anticoagulant use
*   Systolic blood pressure, mmHg
*   Diastolic blood pressure, mmHg
*   PT-INR
*   Time from onset to baseline CT scan, h

In [ ]:
path_tabular_expansion_train = "PATH"
path_tabular_no_expansion_train = "PATH"
path_tabular_expansion_test = "PATH"
path_tabular_no_expansion_test = "PATH"

## Read clinical variables from CSV files

In [ ]:
df_expansion_train = pd.read_csv(path_tabular_expansion_train)
df_no_expansion_train = pd.read_csv(path_tabular_no_expansion_train)
df_expansion_test = pd.read_csv(path_tabular_expansion_test)
df_no_expansion_test = pd.read_csv(path_tabular_no_expansion_test)

## Standardize continuous variables

In [ ]:
scaler = StandardScaler()

scaler.fit(pd.concat([df_expansion_train[["Systolic blood pressure, mmHg"]], df_no_expansion_train[["Systolic blood pressure, mmHg"]]]))
df_expansion_train[["Systolic blood pressure, mmHg"]] = scaler.transform(df_expansion_train[["Systolic blood pressure, mmHg"]])
df_no_expansion_train[["Systolic blood pressure, mmHg"]] = scaler.transform(df_no_expansion_train[["Systolic blood pressure, mmHg"]])
df_expansion_test[["Systolic blood pressure, mmHg"]] = scaler.transform(df_expansion_test[["Systolic blood pressure, mmHg"]])
df_no_expansion_test[["Systolic blood pressure, mmHg"]] = scaler.transform(df_no_expansion_test[["Systolic blood pressure, mmHg"]])

scaler.fit(pd.concat([df_expansion_train[["Diastolic blood pressure, mmHg"]], df_no_expansion_train[["Diastolic blood pressure, mmHg"]]]))
df_expansion_train[["Diastolic blood pressure, mmHg"]] = scaler.transform(df_expansion_train[["Diastolic blood pressure, mmHg"]])
df_no_expansion_train[["Diastolic blood pressure, mmHg"]] = scaler.transform(df_no_expansion_train[["Diastolic blood pressure, mmHg"]])
df_expansion_test[["Diastolic blood pressure, mmHg"]] = scaler.transform(df_expansion_test[["Diastolic blood pressure, mmHg"]])
df_no_expansion_test[["Diastolic blood pressure, mmHg"]] = scaler.transform(df_no_expansion_test[["Diastolic blood pressure, mmHg"]])

scaler.fit(pd.concat([df_expansion_train[["PT-INR"]], df_no_expansion_train[["PT-INR"]]]))
df_expansion_train[["PT-INR"]] = scaler.transform(df_expansion_train[["PT-INR"]])
df_no_expansion_train[["PT-INR"]] = scaler.transform(df_no_expansion_train[["PT-INR"]])
df_expansion_test[["PT-INR"]] = scaler.transform(df_expansion_test[["PT-INR"]])
df_no_expansion_test[["PT-INR"]] = scaler.transform(df_no_expansion_test[["PT-INR"]])

scaler.fit(pd.concat([df_expansion_train[["Time from onset to baseline CT scan, h"]], df_no_expansion_train[["Time from onset to baseline CT scan, h"]]]))
df_expansion_train[["Time from onset to baseline CT scan, h"]] = scaler.transform(df_expansion_train[["Time from onset to baseline CT scan, h"]])
df_no_expansion_train[["Time from onset to baseline CT scan, h"]] = scaler.transform(df_no_expansion_train[["Time from onset to baseline CT scan, h"]])
df_expansion_test[["Time from onset to baseline CT scan, h"]] = scaler.transform(df_expansion_test[["Time from onset to baseline CT scan, h"]])
df_no_expansion_test[["Time from onset to baseline CT scan, h"]] = scaler.transform(df_no_expansion_test[["Time from onset to baseline CT scan, h"]])

## Encode strings as numbers

In [ ]:
df_expansion_train["Anticoagulant use"] = pd.factorize(df_expansion_train["Anticoagulant use"])[0]
df_no_expansion_train["Anticoagulant use"] = pd.factorize(df_no_expansion_train["Anticoagulant use"])[0]
df_expansion_test["Anticoagulant use"] = pd.factorize(df_expansion_test["Anticoagulant use"])[0]
df_no_expansion_test["Anticoagulant use"] = pd.factorize(df_no_expansion_test["Anticoagulant use"])[0]

## Convert DataFrame to NumPy array (clinical variables)

In [ ]:
tabular_expansion_train_val = df_expansion_train.to_numpy().astype("float32")
tabular_no_expansion_train_val = df_no_expansion_train.to_numpy().astype("float32")
tabular_expansion_test = df_expansion_test.to_numpy().astype("float32")
tabular_no_expansion_test = df_no_expansion_test.to_numpy().astype("float32")

# **Preprocessing for CT images**

## Function: Read images and a header (metadata) from a NIfTI file

In [ ]:
def read_image_header(path):
    load = nib.load(path)
    image = load.get_fdata()
    header = load.header
    return image, header

## Functions: Preprocessing for original images

In [ ]:
# scale the CT density (Hounsfield Units) of the images after extracting brain and hematoma
def density_scaling(image):
    image[image < 0] = 0  # use the area with Hounsfield Units between 0 and 100,
    image[image > 100] = 0  # which includes brain and hematoma
    image = image / 100
    return image

# reslice and align the number of slices
def reslice_and_align_slice(image, header, new_spacing=2, new_slices=80):
    # reslice - set new spacing (new_spacing), with which reslice the images
    z_axis_spacing = header['pixdim'][3].round(2)  # get the image spacing from the header
    spacing_factor = z_axis_spacing / new_spacing
    image = ndimage.zoom(image, (1, 1, spacing_factor))

    # align the number of slices - set the new number of slices (new_slices)
    slices = header['dim'][3]  # get the number of slices from the header
    slices_resliced = slices * spacing_factor  # culculate the number of slices after reslicing
    slices_resliced_ = int(slices_resliced.round(0))
    if slices_resliced_ > new_slices:  # eliminate redundant slices without brain to match the new number of slices
        num_reduce = slices_resliced_ - new_slices
        image = np.delete(image, np.s_[:num_reduce], 2)
        return image
    elif slices_resliced_ < new_slices:  # add slices with values of zero to match the new number of slices
        num_add = new_slices - slices_resliced_
        ndarray_zero = np.zeros((image.shape[0], image.shape[1], num_add))
        image = np.concatenate([ndarray_zero, image], 2)
        return image
    else:
        return image

# unify the pixel size
def unify(image, header):
    # calculate the magnification for resizing the images assuming the new pixel size is 0.5 x 0.5
    x_pixel = header['pixdim'][1]  # Since the pixel is square, only the x-axis of the pixel is calculated.
    magnification = x_pixel / 0.5
    # unify
    image = ndimage.zoom(image, (magnification, magnification, 1))
    # padding to restore the image shape to '512 x 512 x 80' after the unification
    padding_0 = int((513 - image.shape[0]) / 2)
    padding_1 = int((513 - image.shape[1]) / 2)
    ndarray_zero = np.zeros((padding_0, image.shape[1], image.shape[2]))  # anterior of the brain
    image = np.concatenate([ndarray_zero, image], 0)
    ndarray_zero = np.zeros((padding_0, image.shape[1], image.shape[2]))  # posterior of the brain
    image = np.concatenate([image, ndarray_zero], 0)
    ndarray_zero = np.zeros((image.shape[0], padding_1, image.shape[2]))  # right of the brain
    image = np.concatenate([ndarray_zero, image], 1)
    ndarray_zero = np.zeros((image.shape[0], padding_1, image.shape[2]))  # left of the brain
    image = np.concatenate([image, ndarray_zero], 1)
    image[image < 0] = 0  # During the unification process, some pixel values may become
    image[image > 1] = 1  # slightly lower than 0 or higher than 1, so the excess are aligned to 0 or 1.
    return image

# resize the images
def resize(image,
           image_size=256  # the image size to be created
           ):
    # resize the image size '512 x 512' to the size specified by the parameter
    image = ndimage.zoom(image, (image_size/512, image_size/512, 1))
    image[image < 0] = 0  # During the resizing process, some pixel values may become
    image[image > 1] = 1  # slightly lower than 0 or higher than 1, so the excess are aligned to 0 or 1.
    return image

# get processed images using above defined functions
def get_processed_images(path):
    image, header = read_image_header(path)
    image = density_scaling(image)
    image = reslice_and_align_slice(image, header)
    image = unify(image, header)
    image = resize(image)
    # rotate the images by 90 degrees
    image = ndimage.rotate(image, 90, reshape=False)
    return image.round(10)

## Functions: Preprocessing for masked images

In [ ]:
# reslice and align the number of slices
def reslice_and_align_slice_mask(image, header, new_spacing=2, new_slices=80):
    # reslice - set new spacing (new_spacing), with which reslice the images
    z_axis_spacing = header['pixdim'][3].round(2)  # get the image spacing from the header
    spacing_factor = z_axis_spacing / new_spacing
    image = ndimage.zoom(image, (1, 1, spacing_factor))
    image[image < 0.2] = 0
    image[image >= 0.2] = 1

    # align the number of slices - set the new number of slices (new_slices)
    slices = header['dim'][3]  # get the number of slices from the header
    slices_resliced = slices * spacing_factor  # culculate the number of slices after reslicing
    slices_resliced_ = int(slices_resliced.round(0))
    if slices_resliced_ > new_slices:  # eliminate redundant slices without brain to match the new number of slices
        num_reduce = slices_resliced_ - new_slices
        image = np.delete(image, np.s_[:num_reduce], 2)
        return image
    elif slices_resliced_ < new_slices:  # add slices with values of zero to match the new number of slices
        num_add = new_slices - slices_resliced_
        ndarray_zero = np.zeros((image.shape[0], image.shape[1], num_add))
        image = np.concatenate([ndarray_zero, image], 2)
        return image
    else:
        return image

# unify the pixel size
def unify_mask(image, header):
    # calculate the magnification for resizing the images assuming the new pixel size is 0.5 x 0.5
    x_pixel = header['pixdim'][1]  # Since the pixel is square, only the x-axis of the pixel is calculated.
    magnification = x_pixel / 0.5
    # unify
    image = ndimage.zoom(image, (magnification, magnification, 1))
    # padding to restore the image shape to '512 x 512 x 80' after the unification
    padding_0 = int((513 - image.shape[0]) / 2)
    padding_1 = int((513 - image.shape[1]) / 2)
    ndarray_zero = np.zeros((padding_0, image.shape[1], image.shape[2]))  # anterior of the brain
    image = np.concatenate([ndarray_zero, image], 0)
    ndarray_zero = np.zeros((padding_0, image.shape[1], image.shape[2]))  # posterior of the brain
    image = np.concatenate([image, ndarray_zero], 0)
    ndarray_zero = np.zeros((image.shape[0], padding_1, image.shape[2]))  # right of the brain
    image = np.concatenate([ndarray_zero, image], 1)
    ndarray_zero = np.zeros((image.shape[0], padding_1, image.shape[2]))  # left of the brain
    image = np.concatenate([image, ndarray_zero], 1)
    image[image < 0.2] = 0   # after the unification, the values of the images are no longer binary,
    image[image >= 0.2] = 1  # therefore, binarize with a threshold of 0.2
    return image

# resize the images
def resize_mask(image,
                image_size=256  # the image size to be created
               ):
    # resize the image size '512 x 512' to the size specified by the parameter
    image = ndimage.zoom(image, (image_size/512, image_size/512, 1))
    image[image < 0.2] = 0   # after resizing, the values of the images are no longer binary,
    image[image >= 0.2] = 1  # therefore, binarize with a threshold of 0.2
    return image

# get processed images using above defined functions
def get_processed_images_mask(path):
    image, header = read_image_header(path)
    image = reslice_and_align_slice_mask(image, header)
    image = unify_mask(image, header)
    image = resize_mask(image)
    # rotate the images by 90 degrees
    image = ndimage.rotate(image, 90, reshape=False)
    return image.round(10)

## Function: Extract hematoma only

In [ ]:
def extract_hematoma(path, path_mask):
    list = [os.path.join(path, i) for i in sorted(os.listdir(path))]
    original = np.array([get_processed_images(j) for j in tqdm(list)])
    original = original.astype('float32')
    list_mask = [os.path.join(path_mask, i) for i in sorted(os.listdir(path_mask))]
    mask = np.array([get_processed_images_mask(j) for j in tqdm(list_mask)])
    mask = mask.astype('float32')
    hematoma = original - 1 + mask
    hematoma[hematoma <= 0] = 0
    return hematoma

## Specify the paths where NIfTI files are stored

The NIfTI files should be stored in each directory according to the following classification method.
*   training and validation, expansion, original CT images
*   training and validation, expansion, masked images
*   training and validation, no expansion, original CT images
*   training and validation, no expansion, masked images
*   test, expansion, original CT images
*   test, expansion, masked images
*   test, no expansion, original CT images
*   test, no expansion, masked images

In [ ]:
path_image_expansion_train_val = "PATH"
path_image_expansion_train_val_mask = "PATH"
path_image_no_expansion_train_val = "PATH"
path_image_no_expansion_train_val_mask = "PATH"
path_image_expansion_test = "PATH"
path_image_expansion_test_mask = "PATH"
path_image_no_expansion_test = "PATH"
path_image_no_expansion_test_mask = "PATH"

## Create image data arrays

In [ ]:
image_expansion_train_val = extract_hematoma(path_image_expansion_train_val, path_image_expansion_train_val_mask)
image_no_expansion_train_val = extract_hematoma(path_image_no_expansion_train_val, path_image_no_expansion_train_val_mask)
image_expansion_test = extract_hematoma(path_image_expansion_test, path_image_expansion_test_mask)
image_no_expansion_test = extract_hematoma(path_image_no_expansion_test, path_image_no_expansion_test_mask)

# **Preprocessing for both clinical variables and CT images**

## Store arrays of clinical variables and CT images in the same array

In [ ]:
expansion_train_val = [[tabular_expansion_train_val[i], image_expansion_train_val[i]] for i in range(len(tabular_expansion_train_val))]
no_expansion_train_val = [[tabular_no_expansion_train_val[i], image_no_expansion_train_val[i]] for i in range(len(tabular_no_expansion_train_val))]

## Shuffle the data in the arrays

In [ ]:
random.seed(42)
np.random.shuffle(expansion_train_val)
np.random.shuffle(no_expansion_train_val)

## 70% were assigned to the training set and the rest to the validation set

In [ ]:
expansion_train = expansion_train_val[:int(len(expansion_train_val)*0.7)]
expansion_val = expansion_train_val[int(len(expansion_train_val)*0.7):]
no_expansion_train = no_expansion_train_val[:int(len(no_expansion_train_val)*0.7)]
no_expansion_val = no_expansion_train_val[int(len(no_expansion_train_val)*0.7):]

## 	Divide the array into the arrays for clinical variables and CT images

In [ ]:
tabular_expansion_train = np.array([expansion_train[i][0] for i in range(len(expansion_train))])
tabular_expansion_val = np.array([expansion_val[i][0] for i in range(len(expansion_val))])
tabular_no_expansion_train = np.array([no_expansion_train[i][0] for i in range(len(no_expansion_train))])
tabular_no_expansion_val = np.array([no_expansion_val[i][0] for i in range(len(no_expansion_val))])
image_expansion_train = np.array([expansion_train[i][1] for i in range(len(expansion_train))])
image_expansion_val = np.array([expansion_val[i][1] for i in range(len(expansion_val))])
image_no_expansion_train = np.array([no_expansion_train[i][1] for i in range(len(no_expansion_train))])
image_no_expansion_val = np.array([no_expansion_val[i][1] for i in range(len(no_expansion_val))])

## Data augmentation by rotating images

In [ ]:
from tqdm import tqdm
from scipy import ndimage
angles = [30]
for j in range(len(angles)):
    for i in tqdm(range(len(image_expansion_train))):
        rotation = ndimage.rotate(image_expansion_train[i], angles[j], reshape=False)
        rotation[rotation < 0] = 0
        rotation[rotation > 1] = 1
        rotation = np.expand_dims(rotation, axis=0)
        image_expansion_train = np.concatenate((image_expansion_train, rotation), axis=0)
        tabular_expansion_train = np.concatenate((tabular_expansion_train, np.expand_dims(tabular_expansion_train[i], axis=0)), axis=0)

## Data augmentation by flipping images

In [ ]:
image_expansion_train = np.concatenate((image_expansion_train, np.flip(image_expansion_train, 2)), axis=0)
tabular_expansion_train = np.concatenate((tabular_expansion_train, tabular_expansion_train), axis=0)

## Labeling

In [ ]:
label_expansion_train = np.array([1 for i in range(len(tabular_expansion_train))])
label_expansion_val = np.array([1 for i in range(len(tabular_expansion_val))])
label_no_expansion_train = np.array([0 for i in range(len(tabular_no_expansion_train))])
label_no_expansion_val = np.array([0 for i in range(len(tabular_no_expansion_val))])
label_expansion_test = np.array([1 for i in range(len(tabular_expansion_test))])
label_no_expansion_test = np.array([0 for i in range(len(tabular_no_expansion_test))])

## Concatenate expansion and no expansion arrays

In [ ]:
tabular_x_train = np.concatenate((tabular_expansion_train, tabular_no_expansion_train), axis=0)
tabular_x_val = np.concatenate((tabular_expansion_val, tabular_no_expansion_val), axis=0)
image_x_train = np.concatenate((image_expansion_train, image_no_expansion_train), axis=0)
image_x_val = np.concatenate((image_expansion_val, image_no_expansion_val), axis=0)
y_train = np.concatenate((label_expansion_train, label_no_expansion_train), axis=0)
y_val = np.concatenate((label_expansion_val, label_no_expansion_val), axis=0)
tabular_x_test = np.concatenate((tabular_expansion_test, tabular_no_expansion_test), axis=0)
image_x_test = np.concatenate((image_expansion_test, image_no_expansion_test), axis=0)
y_test = np.concatenate((label_expansion_test, label_no_expansion_test), axis=0)

## Add a dimension to image arrays

In [ ]:
image_x_train = tf.expand_dims(image_x_train, axis=4)
image_x_val = tf.expand_dims(image_x_val, axis=4)
image_x_test = tf.expand_dims(image_x_test, axis=4)

#  **Training and validation**

## Model architecture

In [ ]:
def model_multimodal():

  inputs_tabular = keras.Input(len(tabular_expansion_train[1]))
  inputs_image = keras.Input((image_expansion_train.shape[1], image_expansion_train.shape[2], image_expansion_train.shape[3], 1))

  t = layers.Dense(units=512, kernel_initializer="he_normal")(inputs_tabular)
  t = layers.BatchNormalization()(t)
  t = layers.Activation("relu")(t)
  t = layers.Dropout(0.3)(t)

  t = layers.Dense(units=256, kernel_initializer="he_normal")(t)
  t = layers.BatchNormalization()(t)
  t = layers.Activation("relu")(t)
  t = layers.Dropout(0.3)(t)

  i = layers.Conv3D(filters=64, kernel_size=(19,19,7), kernel_initializer="he_normal")(inputs_image)
  i = layers.BatchNormalization()(i)
  i = layers.Activation("relu")(i)
  i = layers.MaxPool3D(pool_size=2)(i)

  i = layers.Conv3D(filters=64, kernel_size=(19,19,7), kernel_initializer="he_normal")(i)
  i = layers.BatchNormalization()(i)
  i = layers.Activation("relu")(i)
  i = layers.MaxPool3D(pool_size=2)(i)

  i = layers.Conv3D(filters=128, kernel_size=(14,14,5), kernel_initializer="he_normal")(i)
  i = layers.BatchNormalization()(i)
  i = layers.Activation("relu")(i)
  i = layers.MaxPool3D(pool_size=2)(i)

  i = layers.Conv3D(filters=256, kernel_size=(11,11,4), kernel_initializer="he_normal")(i)
  i = layers.BatchNormalization()(i)
  i = layers.Activation("relu")(i)
  i = layers.MaxPool3D(pool_size=2)(i)

  i = layers.GlobalAveragePooling3D()(i)
  i = layers.Dense(units=512, activation="relu", kernel_initializer="he_normal")(i)
  i = layers.Dropout(0.3)(i)

  concat = layers.concatenate([t, i])

  ti = layers.Dense(units=256, kernel_initializer="he_normal")(concat)
  ti = layers.BatchNormalization()(ti)
  ti = layers.Activation("relu")(ti)
  ti = layers.Dropout(0.3)(ti)

  outputs = layers.Dense(units=1, activation="sigmoid")(ti)

  model = keras.Model((inputs_tabular, inputs_image), outputs, name="multimodal")
  return model

tf.random.set_seed(42)
model = model_multimodal()
model.summary()

## Compile and fit model

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["AUC", tf.keras.metrics.Recall()])
checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath=os.path.join('PATH', '{epoch:03d}.h5'))
model.fit((tabular_x_train, image_x_train), y_train, validation_data=((tabular_x_val, image_x_val), y_val), batch_size=2, epochs=70, callbacks=[checkpoint_cb])

# **Test**

## Specify the path to the HFD5 file that had better sensitivity and AUC in the validation

In [ ]:
model.load_weights("PATH")

## Results

In [ ]:
y_prediction = model.predict((tabular_x_test, image_x_test), batch_size=2, verbose=1)
y_prediction[y_prediction < 0.5] = 0
y_prediction[y_prediction >= 0.5] = 1
y_prediction = y_prediction.astype("int64")
result = confusion_matrix(y_test, y_prediction)
sensitivity = recall_score(y_test, y_prediction)
specificity = recall_score(y_test, y_prediction, pos_label=0)
accuracy = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction)
print(result)
print("sensitivity: {:.3f}".format(sensitivity))
print("specificity: {:.3f}".format(specificity))
print("accuracy: {:.3f}".format(accuracy))
print("AUC: {:.3f}".format(auc))